# Pipeline 
- Insertar Imagen
- Procesar Imagen
- Separar en líneas
- Enviar a los distintos OCRs
- Correcciones 
- Decisión Final

## Insertar Imágenes

In [4]:
input_folder = "data/input"
output_folder = "data/output_preprocessing"

# Procesar Imágenes

In [2]:
from Preprocessing.pipeline2 import cleanup_image

cleanup_image(input_folder, output_folder)

Procesada: Image-1.jpg -> data/output\Image-1.png
Procesada: Image-10.jpg -> data/output\Image-10.png
Procesada: Image-100.jpg -> data/output\Image-100.png
Procesada: Image-11.jpg -> data/output\Image-11.png
Procesada: Image-12.jpg -> data/output\Image-12.png
Procesada: Image-13.jpg -> data/output\Image-13.png
Procesada: Image-14.jpg -> data/output\Image-14.png
Procesada: Image-15.jpg -> data/output\Image-15.png
Procesada: Image-16.jpg -> data/output\Image-16.png
Procesada: Image-17.jpg -> data/output\Image-17.png
Procesada: Image-18.jpg -> data/output\Image-18.png
Procesada: Image-19.jpg -> data/output\Image-19.png
Procesada: Image-2.jpg -> data/output\Image-2.png
Procesada: Image-20.jpg -> data/output\Image-20.png
Procesada: Image-21.jpg -> data/output\Image-21.png
Procesada: Image-22.jpg -> data/output\Image-22.png
Procesada: Image-23.jpg -> data/output\Image-23.png
Procesada: Image-24.jpg -> data/output\Image-24.png
Procesada: Image-25.jpg -> data/output\Image-25.png
Procesada: Ima

## Separar en líneas

## Enviar a los distintos OCRs

In [5]:
import os
from typing import List
from Recognition.Kraken.kraken_model import get_transcription

processed_photos = os.listdir(output_folder)

for photo in processed_photos:
    get_transcription(photo)
    
output_ocr = 'data/output_ocr'
transcriptions = os.listdir(output_ocr)
results_from_ocrs:List[List[List[str]]] = []

for text in transcription:
    text.split('$$')
    # En dependencia de como se guarde hay que picar para los diferentes OCR para las diferentes lineas


Image-1.png
Image-10.png
Image-100.png
Image-11.png
Image-12.png
Image-13.png
Image-14.png
Image-15.png
Image-16.png
Image-17.png
Image-18.png
Image-19.png
Image-2.png
Image-20.png
Image-21.png
Image-22.png
Image-23.png
Image-24.png
Image-25.png
Image-26.png
Image-27.png
Image-28.png
Image-29.png
Image-3.png
Image-30.png
Image-31.png
Image-32.png
Image-33.png
Image-34.png
Image-35.png
Image-36.png
Image-37.png
Image-38.png
Image-39.png
Image-4.png
Image-40.png
Image-41.png
Image-42.png
Image-43.png
Image-44.png
Image-45.png
Image-46.png
Image-47.png
Image-48.png
Image-49.png
Image-5.png
Image-50.png
Image-51.png
Image-52.png
Image-53.png
Image-54.png
Image-55.png
Image-56.png
Image-57.png
Image-58.png
Image-59.png
Image-6.png
Image-60.png
Image-61.png
Image-62.png
Image-63.png
Image-64.png
Image-65.png
Image-66.png
Image-67.png
Image-68.png
Image-69.png
Image-7.png
Image-70.png
Image-71.png
Image-72.png
Image-73.png
Image-74.png
Image-75.png
Image-76.png
Image-77.png
Image-78.png
Image

## Correcciones

In [1]:
from serialization.serialization import correction, combine, load_corrector_model
import google.generativeai as genai
from dotenv import load_dotenv

results_from_ocr = [
    [
        "Essta es una prueva de ectraccion de teexto. Connoscida cosa sea a todos los queesta carta uieren como yo don fffffff por la gracia de dios hey de Castiella.",
        "Asta carta fue escrita en el anno del senyor de 1493, y  cosas de gran ymportancia para los reynos."
    ],
    [
        "Essa s una prsueva de etraccion de . Connoda cosa sea a tdos as los qeesta carta ueren como yo don Fferrando por la gracia de d hey de Castiella.",
        "Asta s fue escrita en el ano de seyor de 143, y consigna cossdas de gran ymportana para los reynos."
    ],
]

# Cargar claves de entorno para Gemini
load_dotenv()
GENAI_API_KEY = os.getenv("GENAI_API_KEY")
genai.configure(api_key=GENAI_API_KEY)

spacy_model = "es_core_news_sm"
frequency_dictionary_path = "serialization/spanish_frequency_dictionary.txt"

# Cargar modelo SymSpell y Spacy
nlp, sym_spell = load_corrector_model(
    spacy_model=spacy_model,
    frequency_dictionary_path=frequency_dictionary_path,
    max_edit_distance=3,
    prefix_length=7
)

gemini_model_name="gemini-2.0-flash-exp"

# Crear el sistema de instrucciones para Gemini
chat = genai.GenerativeModel(
    model_name=gemini_model_name,
    system_instruction=[
        """
        Eres un asistente que ayuda a refinar textos. Analiza el siguiente texto original y corregido,
        y reescríbelo en un formato limpio, coherente y gramaticalmente correcto. Los textos son documentos históricos
        extraídos mediante OCR, por lo que pueden contener errores típicos del procesamiento OCR. Los documentos son
        cartas en Cuba de los siglos XV y XVI, manten el español antiguo.
        """
    ]
).start_chat(history=[])


c:\Users\Admin\miniconda3\envs\posprocesamiento\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
results_from_corrections: List[List[str]] = []

for document in results_from_ocr:
    aux: List[str] = []
    for line in document:
        aux.append(correction(line, nlp, sym_spell, chat))
    results_from_corrections.append(aux)
    
print(results_from_corrections)

[['Esta es una prueba de extracción de texto. Conocida cosa sea a todos los que esta carta vieren, como yo, don [nombre ilegible], por la gracia de Dios, [rey/señor] de Castilla.', 'Esta carta fue escrita en el año del Señor de 1493, y trata de cosas de gran importancia para los reinos.'], ['Esta es una prueba de extracción. Conocida cosa sea a todos los que esta carta vieren, como yo, don Ferrando, por la gracia de Dios, [rey/señor] de Castilla.', 'Esta fue escrita en el año del Señor de 1493, y consigna cosas de gran importancia para los reinos.']]


## Decisión Final

In [3]:
result:List[str] = []

# Crear el modelo generativo y configurar el sistema de instrucciones
chat2 = genai.GenerativeModel(
    model_name=gemini_model_name,
    system_instruction="""
    Eres un asistente que ayuda a refinar textos. Analiza las siguientes representaciones de una línea de texto,
    y reescribe solo una en un formato limpio, coherente y gramaticalmente correcto. Los textos son documentos históricos
    extraídos mediante OCR, por lo que pueden contener errores típicos del procesamiento OCR. Los documentos son
    cartas en Cuba de los siglos XV y XVI, manten el español antiguo en tu respuesta.
    """
).start_chat(history=[])

for it in range(len(results_from_ocr[0])):
    aux: List[str] = []
    for i in range(len(results_from_ocr)):
        aux.append(results_from_corrections[i][it])
    
    result.append(combine(aux, chat2))

print("RESULT FINAL")
print(result)

RESULT FINAL
['Conocida cosa sea a todos los que esta carta vieren, como yo, don Ferrando, por la gracia de Dios, [rey/señor] de Castilla.', 'Esta fue escrita en el año del Señor de 1493, y consigna cosas de gran importancia para los reinos.']
